In [1]:
import numpy as np
import torch

In [2]:
inputs = np.array([[73,67,43],[91,88,64],[87,134,58],[102,43,37],[69,96,70]],dtype='float32')
inputs

array([[ 73.,  67.,  43.],
       [ 91.,  88.,  64.],
       [ 87., 134.,  58.],
       [102.,  43.,  37.],
       [ 69.,  96.,  70.]], dtype=float32)

In [3]:
targets = np.array([[56,70],[81,101],[119,133],[22,37],[103,1119]],dtype=f'float32')
targets

array([[  56.,   70.],
       [  81.,  101.],
       [ 119.,  133.],
       [  22.,   37.],
       [ 103., 1119.]], dtype=float32)

In [4]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[  56.,   70.],
        [  81.,  101.],
        [ 119.,  133.],
        [  22.,   37.],
        [ 103., 1119.]])


In [5]:
w = torch.randn(2,3,requires_grad=True)
b= torch.randn(2,requires_grad=True)
print(w)
print(b)

tensor([[-0.5583,  0.4737,  0.2916],
        [-0.3604,  2.4540,  0.8760]], requires_grad=True)
tensor([ 2.0557, -0.4306], requires_grad=True)


In [6]:
def model(x):
    return x @ w.t() + b

In [7]:
preds = model(inputs)
print(preds)

tensor([[  5.5794, 175.3486],
        [ 11.6023, 238.7917],
        [ 33.8776, 347.8629],
        [-23.7305, 100.7454],
        [ 29.4244, 271.6079]], grad_fn=<AddBackward0>)


In [8]:
def mse(target,preds):
    diff = preds-target
    return torch.sum(diff*diff)/diff.numel()

In [9]:
loss = mse(targets,preds)
print(loss)

tensor(82049.6562, grad_fn=<DivBackward0>)


In [10]:
#computer gradients
loss.backward()

In [11]:
w.grad

tensor([[-5428.5566, -5984.2520, -3677.7932],
        [-2609.0947, -6126.5889, -6229.6328]])

In [12]:
# #it will not calculate the gradient here.
with torch.no_grad():
    w = w - 1e-5 * w.grad
    b = b - 1e-5 * b.grad 
    w.grad.zero_()
    b.grad.zero_()

AttributeError: 'NoneType' object has no attribute 'zero_'

In [ ]:
# preds = model(inputs)
# loss = mse(preds,targets)
# print(loss)

In [ ]:
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [ ]:
nw,nb

(tensor([[-0.0264, -0.9996,  0.0738],
         [ 0.6890,  1.3516, -1.1693]]),
 tensor([-0.7118, -1.2952]))

In [ ]:
# import torch

# # Assuming you have initialized w and b as torch Tensors
# w = torch.randn(10, requires_grad=True)
# b = torch.randn(1, requires_grad=True)

# # Forward pass and loss computation
# x = torch.randn(10)
# y = torch.randn(1)
# output = torch.matmul(x, w) + b
# loss = torch.nn.functional.mse_loss(output, y)

# # Backward pass and gradient computation
# loss.backward()

# # Zero out the gradients
# w.grad.data.zero_()
# b.grad.data.zero_()

# # Print the gradients
# print(w.grad)
# print(b.grad)


In [13]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Define custom dataset
class CustomDataset(Dataset):
    def __init__(self):
        self.x = torch.randn(100, 1)  # Random input features
        self.y = 3 * self.x + 1       # y = 3x + 1

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# Define the linear regression model
class LinearRegression(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)

# Create the model instance
input_size = 1   # Number of features
output_size = 1  # Number of outputs
model = LinearRegression(input_size, output_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Create a custom dataset
dataset = CustomDataset()

#The most important argument of DataLoader constructor is dataset, which indicates a dataset object to load data from.

# Create a data loader
batch_size = 10
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    print(":",epoch)
    for batch_x, batch_y in dataloader:
        # Forward pass
        print(batch_x)
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the loss every 10 epochs
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Test the model with new inputs
test_input = torch.tensor([[0.5], [1.0], [1.5]])
predicted_output = model(test_input)
print("Predicted outputs for test input:")
print(predicted_output)

: 0
tensor([[-0.4379],
        [-1.7098],
        [ 0.2344],
        [-0.4372],
        [ 1.2280],
        [ 0.4308],
        [-0.4889],
        [ 0.3879],
        [-1.5840],
        [-0.1759]])
tensor([[-1.7472],
        [ 0.6198],
        [ 0.3186],
        [-0.0347],
        [-0.1464],
        [-0.9215],
        [ 0.5389],
        [-1.7598],
        [ 0.5020],
        [ 0.5910]])
tensor([[-1.8638],
        [-0.1720],
        [ 0.7008],
        [ 0.4989],
        [ 0.7896],
        [-1.2079],
        [-0.4068],
        [-1.1943],
        [-0.6686],
        [-0.5013]])
tensor([[ 0.3581],
        [-2.0167],
        [ 1.3621],
        [ 1.3055],
        [ 1.6536],
        [ 0.8799],
        [ 1.4686],
        [ 0.5614],
        [ 1.5975],
        [-0.5874]])
tensor([[ 1.4949],
        [ 1.0512],
        [-1.0411],
        [ 0.2907],
        [-0.7468],
        [ 1.5695],
        [-0.4594],
        [ 0.0920],
        [-0.5955],
        [-0.2646]])
tensor([[-0.8820],
        [ 0.9350],
   

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler

# Load the Boston Housing dataset
boston = load_boston()

# print(boston.data)
# Preprocess the dataset
scaler = StandardScaler()
data = scaler.fit_transform(boston.data)
target = boston.target

# Create custom dataset
class CustomDataset(Dataset):
    def __init__(self, data, target):
        self.x = torch.tensor(data, dtype=torch.float32)
        self.y = torch.tensor(target, dtype=torch.float32).view(-1, 1)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# Create the model
model = nn.Linear(data.shape[1], 1)

# Create the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Create the custom dataset and data loader
dataset = CustomDataset(data, target)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    for batch_x, batch_y in dataloader:
        # Forward pass
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the loss every 10 epochs
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Test the model with new inputs
test_input = torch.tensor(data[:5], dtype=torch.float32)
predicted_output = model(test_input)
print("Predicted outputs for test input:")
print(predicted_output)


c:\Users\ASUS\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i

[[6.3200e-03 1.8000e+01 2.3100e+00 ... 1.5300e+01 3.9690e+02 4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9690e+02 9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9283e+02 4.0300e+00]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 5.6400e+00]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9345e+02 6.4800e+00]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 7.8800e+00]]
Epoch [10/100], Loss: 11.2264
Epoch [20/100], Loss: 37.0186
Epoch [30/100], Loss: 18.9747
Epoch [40/100], Loss: 40.8983
Epoch [50/100], Loss: 22.3805
Epoch [60/100], Loss: 9.1215
Epoch [70/100], Loss: 11.9530
Epoch [80/100], Loss: 17.4342
Epoch [90/100], Loss: 24.9084
Epoch [100/100], Loss: 43.3211
Predicted outputs for test input:
tensor([[29.9812],
        [24.8600],
        [30.3553],
        [28.3803],
        [27.7190]], grad_fn=<AddmmBackward0>)


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

# Load the California Housing dataset
california_housing = fetch_california_housing()

# Preprocess the dataset
scaler = StandardScaler()
data = scaler.fit_transform(california_housing.data)
target = california_housing.target

# Create custom dataset
class CustomDataset(Dataset):
    def __init__(self, data, target):
        self.x = torch.tensor(data, dtype=torch.float32)
        self.y = torch.tensor(target, dtype=torch.float32).view(-1, 1)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# Create the model
model = nn.Linear(data.shape[1], 1)

# Create the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001)

# Create the custom dataset and data loader
dataset = CustomDataset(data, target)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    for batch_x, batch_y in dataloader:
        # Forward pass
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the loss every 10 epochs
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Test the model with new inputs
test_input = torch.tensor(data[:5], dtype=torch.float32)
predicted_output = model(test_input)
print("Predicted outputs for test input:")
print(predicted_output)

Epoch [10/100], Loss: 0.5730
Epoch [20/100], Loss: 0.8416
Epoch [30/100], Loss: 0.7204
Epoch [40/100], Loss: 0.5919
Epoch [50/100], Loss: 0.5749
Epoch [60/100], Loss: 0.5111
Epoch [70/100], Loss: 0.5730
Epoch [80/100], Loss: 0.6785
Epoch [90/100], Loss: 0.2913
Epoch [100/100], Loss: 0.2517
Predicted outputs for test input:
tensor([[4.1633],
        [3.9674],
        [3.7032],
        [3.2466],
        [2.3648]], grad_fn=<AddmmBackward0>)
